In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def batch_canny(images):
    canny_list = []
    for i in tqdm(range(len(images))):
        img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
        canny_superposition = np.zeros(shape=(img.shape[0],img.shape[1]),dtype=np.uint16)
        for channel in range(3):
            img_ch = img[:,:,channel]
            canny_ch = cv2.Canny(img_ch, 60, 150)
            canny_superposition =  canny_superposition + (canny_ch).astype(np.uint16)
        canny_superposition = cv2.normalize(canny_superposition, canny_superposition,0,255,cv2.NORM_MINMAX)
        canny_list.append(np.array((canny_superposition).astype(np.uint8)).reshape([img.shape[0],img.shape[1],1]))
    canny_list = np.array(canny_list,dtype = float32)
    return canny_list


In [3]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = '../train/'
    TRAIN_SUB_DIR = '../subsample/'
    TRAIN_GAN = '../../image_gan/'
    TEST_DIR = '../../test/'

    # use this for full dataset
#     train_images_original = [TRAIN_ORIGINAL_DIR + i for i in os.listdir(TRAIN_ORIGINAL_DIR)]
#     train_images_sub = [TRAIN_SUB_DIR + i for i in os.listdir(TRAIN_SUB_DIR)]
#     train_images = train_images_original + train_images_sub
    train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images = train_images_gan
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "."
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255
    testImage = testImage.astype(np.float32) / 255
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [4]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

trainImage_canny = batch_canny(trainImage)
testImage_canny = batch_canny(testImage)

nb_classes = 2

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)
np.save('../trainImage_canny.npy', trainImage_canny)
np.save('../testImage_canny.npy', testImage_canny)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] trainImage_canny matrix: {:.2f}MB".format(
    (trainImage_canny.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_canny matrix: {:.4f}MB".format(
    (testImage_canny.nbytes) / (1024 * 1000.0)))

[INFO] handling images...
[INFO] processed 1000/6000
[INFO] processed 2000/6000
[INFO] processed 3000/6000
[INFO] processed 4000/6000
[INFO] processed 5000/6000
[INFO] processed 6000/6000
[INFO] processed 154/154
(6000, 128, 128, 3)


100%|██████████| 154/154 [00:01<00:00, 79.33it/s]


[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0.

In [5]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_CANNY_C%d_D%d_Kernel(%d,%d)' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1])    

        # train
        #设置一个全局的计数器
#         self.global_step = tf.Variable(0, trainable=False)
#        self.lr = tf.train.exponential_decay(0.0002,  self.global_step, 500, 0.95, staircase=True)
        self.lr = 0.001
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        self.train_x_canny = np.load('../trainImage_canny.npy')
        self.test_x_canny = np.load('../testImage_canny.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_canny, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_canny",x_canny.get_shape()) # 128, 128, 1
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_canny,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_canny = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, 1], 
                                name='x_canny')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_canny, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.x_canny, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_canny = self.train_x_canny[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_canny = shuffled_set_canny[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_canny: batch_x_canny,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
                    # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_canny_test = self.test_x_canny[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_canny: batch_x_canny_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
        
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

                # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_canny_test = self.test_x_canny[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_canny: batch_x_canny_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_canny_test = self.test_x_canny[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_canny: batch_x_canny_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [6]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
   
# Avg per epoch ptime: 15.45, total 50 epochs ptime: 763.43
#  [*] Training finished!
#  [*] Best Epoch:  5 , Accuracy:  0.9199999570846558
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_CANNY_C5_D1_Kernel(3,3)/CNN_CANNY_C5_D1_Kernel(3,3)-5
#  [*] Finished testing Best Epoch: 5 , accuracy:  0.9200000166893005 !


CNN:x (100, 128, 128, 3)
CNN:x_canny (100, 128, 128, 1)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_canny (100, 128, 128, 1)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_1_2/kernel:0 (float32_ref 3x3x1x32) [288, bytes: 1152]
cnn/conv_1

Epoch: [ 3] [  37/  60] time: 27.5823, loss: 0.02937135
Epoch: [ 3] [  38/  60] time: 27.8226, loss: 0.10222408
Epoch: [ 3] [  39/  60] time: 28.0626, loss: 0.06052536
Epoch: [ 3] [  40/  60] time: 28.3008, loss: 0.03372791
Epoch: [ 3] [  41/  60] time: 28.5379, loss: 0.08426842
Epoch: [ 3] [  42/  60] time: 28.7763, loss: 0.04910346
Epoch: [ 3] [  43/  60] time: 29.0138, loss: 0.05235939
Epoch: [ 3] [  44/  60] time: 29.2514, loss: 0.07459735
Epoch: [ 3] [  45/  60] time: 29.4897, loss: 0.03476496
Epoch: [ 3] [  46/  60] time: 29.7292, loss: 0.05266124
Epoch: [ 3] [  47/  60] time: 29.9673, loss: 0.06252980
Epoch: [ 3] [  48/  60] time: 30.2069, loss: 0.10981078
Epoch: [ 3] [  49/  60] time: 30.4451, loss: 0.03788959
Epoch: [ 3] [  50/  60] time: 30.6826, loss: 0.01390643
Epoch: [ 3] [  51/  60] time: 30.9198, loss: 0.03719718
Epoch: [ 3] [  52/  60] time: 31.1588, loss: 0.01156404
Epoch: [ 3] [  53/  60] time: 31.3974, loss: 0.03654322
Epoch: [ 3] [  54/  60] time: 31.6356, loss: 0.0

Epoch: [ 6] [   1/  60] time: 65.4870, loss: 0.03533339
Epoch: [ 6] [   2/  60] time: 65.7272, loss: 0.00630656
Epoch: [ 6] [   3/  60] time: 65.9651, loss: 0.02059356
Epoch: [ 6] [   4/  60] time: 66.2046, loss: 0.01174976
Epoch: [ 6] [   5/  60] time: 66.4420, loss: 0.01189705
Epoch: [ 6] [   6/  60] time: 66.6811, loss: 0.00691202
Epoch: [ 6] [   7/  60] time: 66.9198, loss: 0.04474347
Epoch: [ 6] [   8/  60] time: 67.1600, loss: 0.00557459
Epoch: [ 6] [   9/  60] time: 67.3987, loss: 0.00912334
Epoch: [ 6] [  10/  60] time: 67.6371, loss: 0.01761880
Epoch: [ 6] [  11/  60] time: 67.8777, loss: 0.00824552
Epoch: [ 6] [  12/  60] time: 68.1159, loss: 0.02996678
Epoch: [ 6] [  13/  60] time: 68.3548, loss: 0.01663776
Epoch: [ 6] [  14/  60] time: 68.5926, loss: 0.02657516
Epoch: [ 6] [  15/  60] time: 68.8320, loss: 0.00145575
Epoch: [ 6] [  16/  60] time: 69.0707, loss: 0.01164136
Epoch: [ 6] [  17/  60] time: 69.3085, loss: 0.01244289
Epoch: [ 6] [  18/  60] time: 69.5476, loss: 0.0

Epoch: [ 8] [  26/  60] time: 102.5128, loss: 0.00914284
Epoch: [ 8] [  27/  60] time: 102.7562, loss: 0.01717425
Epoch: [ 8] [  28/  60] time: 102.9951, loss: 0.00987453
Epoch: [ 8] [  29/  60] time: 103.2359, loss: 0.00221748
Epoch: [ 8] [  30/  60] time: 103.4744, loss: 0.01608286
Epoch: [ 8] [  31/  60] time: 103.7145, loss: 0.00654970
Epoch: [ 8] [  32/  60] time: 103.9526, loss: 0.00053313
Epoch: [ 8] [  33/  60] time: 104.1926, loss: 0.04430823
Epoch: [ 8] [  34/  60] time: 104.4313, loss: 0.00605630
Epoch: [ 8] [  35/  60] time: 104.6708, loss: 0.00412572
Epoch: [ 8] [  36/  60] time: 104.9091, loss: 0.00047600
Epoch: [ 8] [  37/  60] time: 105.1475, loss: 0.12538783
Epoch: [ 8] [  38/  60] time: 105.3858, loss: 0.03117143
Epoch: [ 8] [  39/  60] time: 105.6240, loss: 0.03643085
Epoch: [ 8] [  40/  60] time: 105.8638, loss: 0.04922412
Epoch: [ 8] [  41/  60] time: 106.1030, loss: 0.02280703
Epoch: [ 8] [  42/  60] time: 106.3427, loss: 0.00351469
Epoch: [ 8] [  43/  60] time: 1

Epoch: [10] [  48/  60] time: 138.8354, loss: 0.03124845
Epoch: [10] [  49/  60] time: 139.0744, loss: 0.00051653
Epoch: [10] [  50/  60] time: 139.3159, loss: 0.00816282
Epoch: [10] [  51/  60] time: 139.5544, loss: 0.02100883
Epoch: [10] [  52/  60] time: 139.7939, loss: 0.07888569
Epoch: [10] [  53/  60] time: 140.0325, loss: 0.03000750
Epoch: [10] [  54/  60] time: 140.2738, loss: 0.01546432
Epoch: [10] [  55/  60] time: 140.5127, loss: 0.06756637
Epoch: [10] [  56/  60] time: 140.7544, loss: 0.03018464
Epoch: [10] [  57/  60] time: 140.9933, loss: 0.13489787
Epoch: [10] [  58/  60] time: 141.2347, loss: 0.00541478
Epoch: [10] [  59/  60] time: 141.4739, loss: 0.03996805
[10/50] - ptime: 15.1347 loss: 0.01574815 acc: 0.33000
Epoch: [11] [   0/  60] time: 142.8325, loss: 0.00540623
Epoch: [11] [   1/  60] time: 143.0722, loss: 0.03445499
Epoch: [11] [   2/  60] time: 143.3120, loss: 0.00593930
Epoch: [11] [   3/  60] time: 143.5514, loss: 0.01179086
Epoch: [11] [   4/  60] time: 143

...

Epoch: [41] [  29/  60] time: 615.9777, loss: 0.00079383
Epoch: [41] [  30/  60] time: 616.2171, loss: 0.00096692
Epoch: [41] [  31/  60] time: 616.4597, loss: 0.03016074
Epoch: [41] [  32/  60] time: 616.7006, loss: 0.00228313
Epoch: [41] [  33/  60] time: 616.9404, loss: 0.00101224
Epoch: [41] [  34/  60] time: 617.1800, loss: 0.00038630
Epoch: [41] [  35/  60] time: 617.4203, loss: 0.00002749
Epoch: [41] [  36/  60] time: 617.6599, loss: 0.00012787
Epoch: [41] [  37/  60] time: 617.8996, loss: 0.01069392
Epoch: [41] [  38/  60] time: 618.1369, loss: 0.00184625
Epoch: [41] [  39/  60] time: 618.3785, loss: 0.00039916
Epoch: [41] [  40/  60] time: 618.6175, loss: 0.00353908
Epoch: [41] [  41/  60] time: 618.8587, loss: 0.00495094
Epoch: [41] [  42/  60] time: 619.0972, loss: 0.02101634
Epoch: [41] [  43/  60] time: 619.3382, loss: 0.00003504
Epoch: [41] [  44/  60] time: 619.5774, loss: 0.00021913
Epoch: [41] [  45/  60] time: 619.8177, loss: 0.03639547
Epoch: [41] [  46/  60] time: 6

Epoch: [43] [  51/  60] time: 652.3327, loss: 0.00827339
Epoch: [43] [  52/  60] time: 652.5731, loss: 0.00559860
Epoch: [43] [  53/  60] time: 652.8163, loss: 0.00000378
Epoch: [43] [  54/  60] time: 653.0564, loss: 0.00047114
Epoch: [43] [  55/  60] time: 653.2966, loss: 0.00001402
Epoch: [43] [  56/  60] time: 653.5383, loss: 0.00004352
Epoch: [43] [  57/  60] time: 653.7794, loss: 0.00010973
Epoch: [43] [  58/  60] time: 654.0183, loss: 0.00006836
Epoch: [43] [  59/  60] time: 654.2590, loss: 0.00017729
[43/50] - ptime: 15.1365 loss: 0.00102011 acc: 0.89000
Epoch: [44] [   0/  60] time: 655.5777, loss: 0.01558737
Epoch: [44] [   1/  60] time: 655.8172, loss: 0.00058836
Epoch: [44] [   2/  60] time: 656.0568, loss: 0.00002901
Epoch: [44] [   3/  60] time: 656.2968, loss: 0.00001094
Epoch: [44] [   4/  60] time: 656.5380, loss: 0.00041789
Epoch: [44] [   5/  60] time: 656.7794, loss: 0.00068665
Epoch: [44] [   6/  60] time: 657.0135, loss: 0.00243623
Epoch: [44] [   7/  60] time: 657

Epoch: [46] [  12/  60] time: 689.5011, loss: 0.00060074
Epoch: [46] [  13/  60] time: 689.7420, loss: 0.00001522
Epoch: [46] [  14/  60] time: 689.9819, loss: 0.00003781
Epoch: [46] [  15/  60] time: 690.2212, loss: 0.00000188
Epoch: [46] [  16/  60] time: 690.4619, loss: 0.00006359
Epoch: [46] [  17/  60] time: 690.7032, loss: 0.00032154
Epoch: [46] [  18/  60] time: 690.9377, loss: 0.00000349
Epoch: [46] [  19/  60] time: 691.1765, loss: 0.00002404
Epoch: [46] [  20/  60] time: 691.4157, loss: 0.00004234
Epoch: [46] [  21/  60] time: 691.6552, loss: 0.00059802
Epoch: [46] [  22/  60] time: 691.8941, loss: 0.00000007
Epoch: [46] [  23/  60] time: 692.1337, loss: 0.00028115
Epoch: [46] [  24/  60] time: 692.3745, loss: 0.00005310
Epoch: [46] [  25/  60] time: 692.6151, loss: 0.00003144
Epoch: [46] [  26/  60] time: 692.8561, loss: 0.00010942
Epoch: [46] [  27/  60] time: 693.0963, loss: 0.00003038
Epoch: [46] [  28/  60] time: 693.3359, loss: 0.00007405
Epoch: [46] [  29/  60] time: 6

Epoch: [48] [  34/  60] time: 725.9073, loss: 0.00000097
Epoch: [48] [  35/  60] time: 726.1467, loss: 0.00002037
Epoch: [48] [  36/  60] time: 726.3870, loss: 0.00001121
Epoch: [48] [  37/  60] time: 726.6282, loss: 0.00000065
Epoch: [48] [  38/  60] time: 726.8684, loss: 0.00002488
Epoch: [48] [  39/  60] time: 727.1074, loss: 0.00000190
Epoch: [48] [  40/  60] time: 727.3487, loss: 0.00013672
Epoch: [48] [  41/  60] time: 727.5908, loss: 0.00000477
Epoch: [48] [  42/  60] time: 727.8308, loss: 0.00000620
Epoch: [48] [  43/  60] time: 728.0697, loss: 0.00000200
Epoch: [48] [  44/  60] time: 728.3091, loss: 0.00006095
Epoch: [48] [  45/  60] time: 728.5530, loss: 0.00000035
Epoch: [48] [  46/  60] time: 728.7931, loss: 0.00000940
Epoch: [48] [  47/  60] time: 729.0326, loss: 0.00001193
Epoch: [48] [  48/  60] time: 729.2748, loss: 0.00003309
Epoch: [48] [  49/  60] time: 729.5159, loss: 0.00003160
Epoch: [48] [  50/  60] time: 729.7572, loss: 0.00016714
Epoch: [48] [  51/  60] time: 7

Epoch: [50] [  56/  60] time: 762.2707, loss: 0.00026923
Epoch: [50] [  57/  60] time: 762.5094, loss: 0.00001292
Epoch: [50] [  58/  60] time: 762.7543, loss: 0.00000062
Epoch: [50] [  59/  60] time: 762.9946, loss: 0.00000138
[50/50] - ptime: 15.1470 loss: 0.00016099 acc: 0.75000
Avg per epoch ptime: 15.45, total 50 epochs ptime: 763.43
 [*] Training finished!
 [*] Best Epoch:  5 , Accuracy:  0.9199999570846558
INFO:tensorflow:Restoring parameters from checkpoint/CNN_CANNY_C5_D1_Kernel(3,3)/CNN_CANNY_C5_D1_Kernel(3,3)-5
 [*] Finished testing Best Epoch: 5 , accuracy:  0.9200000166893005 !


In [7]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放